# Senior Top 10

Created by Michael George (AKA Logiqx)

Note: This can only be run after "Overs 40s" and "Percentile Ranks"

Link: https://www.speedsolving.com/forum/showthread.php?54128-How-fast-are-the-over-40-s-in-competitions

In [72]:
# List of recognised events
events = \
[
    ('333', '3x3x3 Cube', '10', 'time', 180),
    ('222', '2x2x2 Cube', '20', 'time', 60),
    ('444', '4x4x4 Cube', '30', 'time', 180),
    ('555', '5x5x5 Cube', '40', 'time', 240),
    ('666', '6x6x6 Cube', '50', 'time', 360),
    ('777', '7x7x7 Cube', '60', 'time', 540),
    ('333bf', '3x3x3 Blindfolded', '70', 'time', 240),
    ('333fm', '3x3x3 Fewest Moves', '80', 'number', 60),
    ('333oh', '3x3x3 One-Handed', '90', 'time', 120),
    ('333ft', '3x3x3 With Feet', '100', 'time', 240),
    ('clock', 'Clock', '110', 'time', 60),
    ('minx', 'Megaminx', '120', 'time', 240),
    ('pyram', 'Pyraminx', '130', 'time', 60),
    ('skewb', 'Skewb', '140', 'time', 60),
    ('sq1', 'Square-1', '150', 'time', 120),
    ('444bf', '4x4x4 Blindfolded', '160', 'time', 0),
    ('555bf', '5x5x5 Blindfolded', '170', 'time', 0),
    ('333mbf', '3x3x3 Multi-Blind', '180', 'multi', 0)
]

# Dictionary of recognised events
eventsDict = {}
for event in events:
    eventsDict[event[0]] = (event[1:])

## Seconds to HH:MM:SS

Intelligently convert seconds to hours, minutes and seconds

In [73]:
def formatTime(seconds):
    if seconds >= 3600:
        return str(seconds / 3600) + ':' + str(seconds % 3600 / 60).zfill(2) + ':' + str(seconds % 60).zfill(2)
    elif seconds >= 60:
        return str(seconds / 60) + ':' + str(seconds % 60).zfill(2)
    else:
        return str(seconds)
    
def decodeTime(result):
    seconds = 0
    parts = result.split(':')

    for part in parts:
        seconds = seconds * 60 + int(part)

    return seconds

## Read Event Results from CSV

Read event data from CSV into memory, prior to processing

In [74]:
import os, csv

class EventResults:
    
    def __init__(self):
        """Initialisise the event results"""
        
        self.basename = None
        self.event = None
        self.results = []
        self.total = 0
        
    def readResults(self, basename, event):
        """Read event results from CSV into memory"""
        
        self.basename = basename
        self.event = event
        self.results = []
        self.total = 0

        # Read rows using the CSV reader
        fn = os.path.join('data', 'public', self.basename, self.event[0] + '.csv')
        with open(fn, 'rb') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for inputRow in csvReader:
                
                # Pack out results with zeros
                while (int(inputRow[0]) > len(self.results)):
                    self.results.append([0, self.total])
                
                count = int(inputRow[1])
                self.total += count
                self.results.append([count, self.total])

## Read Partial Results from CSV

Read event data from CSV into memory, prior to processing

In [75]:
import os, csv

class PartialResults:
    
    def __init__(self):
        """Initialisise the partial results"""
        
        self.basename = None
        self.event = None
        self.results = {}
        self.total = 0
        
    def readResults(self, basename, event):
        """Read event results from CSV into memory"""
        
        self.basename = basename
        self.event = event
        self.results = {}
        self.total = 0

        # Read rows using the CSV reader
        fn = os.path.join('data', 'public', self.basename, self.event[0] + '.csv')
        with open(fn, 'rb') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for inputRow in csvReader:
                
                result = inputRow[5].split('.')[0]
                
                if self.event[3] == 'time':
                    result = decodeTime(result)
                else:
                    result = int(result)
                    
                if result > self.event[4]:
                    result = self.event[4]
                
                if self.results.has_key(result):
                    self.results[result] += [inputRow]
                else:
                    self.results[result] = [inputRow]

## Analyse Results

Process all three sets of results simultaneously

In [76]:
class EventAnalysis:
    
    def __init__(self):
        """Initialisise the event analysis"""
        
        self.event = None
        self.seniorResults = None
        self.partialResults = None
        
    def readResults(self, event):
        """Read event results from CSV into memory"""

        # Skip processing if no cutoff is defined
        if event[4] > 0:
            self.event = event

            self.seniorResults = EventResults()
            self.seniorResults.readResults('senior_averages', self.event)

            self.partialResults = PartialResults()
            self.partialResults.readResults('top_averages', self.event)

    def getHtml(self):
        """Get the HTML for the event"""
        
        html = ''
        
        if self.event:
            html += '<details>'
            html += '<summary>%s</summary>\n' % self.event[1]
            html += '<table>\n'
            html += '<tr>'
            for field in ['#', 'Person(s)', 'Result']:
                html += '<td><b>%s</b></td>' % field
            html += '</tr>\n'

            i = 0
            count = 0
            pos = 1
            while i < len(self.seniorResults.results):
                
                seniorResult = self.seniorResults.results[i]
                
                if self.partialResults.results.has_key(i):
                    knownResults = self.partialResults.results[i]
                else:
                    knownResults = []

                known = len(knownResults)
                unknown = seniorResult[0] - len(knownResults)

                # The result may be a time or count
                if self.event[3] == 'time':
                    result = formatTime(i)
                else:
                    result = str(i)
                    
                if (i >= self.event[4]):
                    result = 'sup-' + result
                else:
                    result +=  '.xx'

                if unknown == 0:
                    for knownResult in knownResults:

                        link = '<a href="https://www.worldcubeassociation.org/results/p.php?i=%s#%s">%s</a>' % \
                                (knownResult[2], self.event[0], knownResult[2]);
                        persons = '%s, %s - %s' % (knownResult[1], knownResult[4], link)

                        html += '<tr>'
                        html += '<td>%s</td>' % pos
                        html += '<td>%s</td>' % persons
                        html += '<td>%s</td>' % knownResult[5]
                        html += '</tr>\n'

                        pos += 1

                else:
                    persons = ''
                    if len(knownResults) > 0:
                        for knownResult in knownResults:
                            if persons:
                                persons += '<br/>'
                            link = '<a href="https://www.worldcubeassociation.org/results/p.php?i=%s#%s">%s</a>' % \
                                    (knownResult[2], self.event[0], knownResult[2]);
                            persons += '%s, %s - %s - %s' % (knownResult[1], knownResult[4], link, knownResult[5])
                        if unknown > 0:
                            persons += '<br/>+ '
                            persons += '%d unknown%s' % (unknown, 's' if unknown > 1 else '')
                    else:
                        if unknown > 1:
                            persons += '%d unknown%s' % (unknown, 's' if unknown > 1 else '')
                        elif unknown == 1:
                            persons += 'Unknown'

                    html += '<tr>'
                    html += '<td>%s</td>' % (pos if seniorResult[0] == 1 else '%d-%d' % (pos, seniorResult[1]))
                    html += '<td>%s</td>' % details
                    html += '<td>%s</td>' % result
                    html += '</tr>\n'

                    pos += seniorResult[0]

                    count = seniorResult[1]
                    
                i += 1

            html += '</table>\n'
            html += '</details>\n\n'
        
        return html

## Analyse Events

Process the events one-by-one

In [77]:
html = '''<h1>Background</h1>
<p>This project started on the <a href="https://www.speedsolving.com/forum/threads/how-fast-are-the-over-40s-in-competitions.54128/">SpeedSolving.com</a> forum and was last updated %s.</p>
<p>The statistics below have been calculated using <a href="sql/senior_aggregates.sql">aggregated data</a> from the WCA database.</p>
\n''' % '2019-02-01'

html += '<h1>%s</h1>\n\n' % 'Official Averages'

for event in events:
    eventAnalysis = EventAnalysis()
    eventAnalysis.readResults(event)
    html += eventAnalysis.getHtml()
    
with open("Senior Rankings.md", 'w') as f:
    f.write(html)